# Object Detection Pipeline - Model Experimentation

This notebook demonstrates the complete pipeline for training different YOLO models on aerial imagery.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-oda/blob/master/notebooks/experiment.ipynb)

In [34]:
import os
import geopandas as gpd
from pathlib import Path
from dl4cv_oda import (
    clean_osm_data,
    clip_labels_to_tiles,
    convert_to_yolo_format,
    create_train_val_split,
    create_yolo_config,
    train_model,
    download_tiles
)

## Configuration

In [35]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

In [36]:
import requests 

if not OSM_FILE.exists():
    response = requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True)
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(response.content)
    print(f"Downloaded file to {OSM_FILE}")


Downloaded file to ../data/raw/kolovai-trees.geojson


## Step 1: Clean OSM Data

In [38]:
if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Processed {count} trees")
else:
    print("Cleaned data already exists")

Cleaned data already exists


## Step 1.5: Download Tiles

Download aerial imagery tiles for the area of interest if not already present.

In [39]:
import shutil 

if not TILES_FILE.exists():
    print("Downloading tiles...")
    
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326,inplace=True)
    min_lon, min_lat, max_lon, max_lat = data.total_bounds
    bbox = [min_lon, min_lat, max_lon, max_lat]    
    ZOOM = 19
    TMS = "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png"
    
    await download_tiles(bbox,ZOOM,TMS,DATA_DIR,'OAM')
    
    


Total tiles fetched: 551
Detected tile scheme: xyz


  0%|          | 0/551 [00:00<?, ?tile/s]

https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/19/6773/293571.png
https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/19/6773/293572.png
https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/19/6773/293573.png
https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/19/6773/293574.png
https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/19/6773/293575.png
https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/19/6773/293576.png
https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/19/6773/293577.png
https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/19/6773/293578.png
https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a1

/Users/krschap/academia/dl4cv-oda/.venv/lib/python3.13/site-packages/rasterio/__init__.py:369: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = get_writer_for_path(path, driver=driver)(
100%|██████████| 551/551 [00:05<00:00, 98.70tile/s] 


## Step 2: Clip Labels to Tiles

Assumes tiles have already been downloaded.

In [40]:
if TILES_FILE.exists():
    stats = clip_labels_to_tiles(
        str(TREES_BOX_FILE), 
        str(TILES_FILE), 
        str(LABELS_DIR)
    )
    print(f"Processed {stats['processed']} tiles, {stats['total_trees']} trees")
else:
    print("Tiles file not found. Run tiling step first.")

Processed 458 tiles, 12337 trees


## Step 3: Convert to YOLO Format

In [41]:
class_mapping = convert_to_yolo_format(
    str(TREES_BOX_FILE),
    str(CHIPS_DIR),
    str(LABELS_DIR),
    str(YOLO_DIR),
    target_species="Coconut",
)

print("Class mapping:", class_mapping)

Class mapping: {'Coconut': 0}


## Step 4: Create Train/Val Split

In [42]:
train_count, val_count = create_train_val_split(
    str(LABELS_DIR),
    str(CHIPS_DIR),
    str(YOLO_DIR),
    train_ratio=0.8,
)

print(f"Train: {train_count} | Val: {val_count}")

/Users/krschap/academia/dl4cv-oda/src/dl4cv_oda/yolo_converter.py:86: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = df.groupby("species", group_keys=False).apply(


Train: 366 | Val: 92


## Step 5: Create YOLO Config

In [7]:
config_file = create_yolo_config(str(YOLO_DIR), class_mapping)
print(f"Config saved to: {config_file}")

Config saved to: ../data/yolo/config.yaml


## Experiment 1: YOLOv8n (nano)

In [8]:
results_n = train_model(
    str(config_file),
    epochs=100,
    imgsz=256,
    batch=16,
    name="coconut_yolov8n",
    model_name="yolov8n.pt",
)

New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/yolo/config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=coconut_yolov8

## Experiment 2: YOLOv8s (small)

In [9]:
results_s = train_model(
    str(config_file),
    epochs=100,
    imgsz=256,
    batch=16,
    name="coconut_yolov8s",
    model_name="yolov8s.pt",
)

New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/yolo/config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=coconut_yolov8

## Experiment 3: YOLOv8m (medium)

In [10]:
results_m = train_model(
    str(config_file),
    epochs=100,
    imgsz=256,
    batch=8,
    name="coconut_yolov8m",
    model_name="yolov8m.pt",
)

New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/yolo/config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=coconut_yolov8m

## Compare Results

Check the `runs/detect/` directory for training results and metrics.

In [11]:
import pandas as pd
from pathlib import Path

runs_dir = Path("runs/detect")
if runs_dir.exists():
    experiments = sorted([d.name for d in runs_dir.iterdir() if d.is_dir()])
    print("Available experiments:")
    for exp in experiments:
        print(f"  - {exp}")
        results_csv = runs_dir / exp / "results.csv"
        if results_csv.exists():
            df = pd.read_csv(results_csv)
            if len(df) > 0:
                last_row = df.iloc[-1]
                print(f"    mAP50: {last_row.get('metrics/mAP50(B)', 'N/A')}")
                print(f"    mAP50-95: {last_row.get('metrics/mAP50-95(B)', 'N/A')}")

Available experiments:
  - coconut_yolov8m
    mAP50: 0.69484
    mAP50-95: 0.27692
  - coconut_yolov8n
    mAP50: 0.678
    mAP50-95: 0.27509
  - coconut_yolov8s
    mAP50: 0.6855
    mAP50-95: 0.27275


## Custom Experiment

Run your own experiment with custom parameters:

In [12]:
MODEL = "yolov8n.pt"
EPOCHS = 50
BATCH = 16
IMG_SIZE = 256
EXPERIMENT_NAME = "custom_experiment"

results_custom = train_model(
    str(config_file),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    name=EXPERIMENT_NAME,
    model_name=MODEL,
)

New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/yolo/config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=custom_experime